In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""

In [3]:
import openai

openai.__version__

# openai.ChatCompletion("hi")

'0.28.0'

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [5]:
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
from langchain.chains.llm import LLMChain
import openai


def schema_text(node_props, rel_props, rels):
    return f"""
  This is the schema representation of the Neo4j database.
  Node properties are the following:
  {node_props}
  Relationship properties are the following:
  {rel_props}
  Relationship point from source to target nodes
  {rels}
  Make sure to respect relationship types and directions
  """


class Neo4jGPTQuery:
    def __init__(self, url, user, password):
        self.driver = GraphDatabase.driver(url, auth=(user, password))
        # openai.api_key = openai_api_key
        # construct schema
        self.schema = self.generate_schema()


    def generate_schema(self):
        node_props = self.query_database(node_properties_query)
        rel_props = self.query_database(rel_properties_query)
        rels = self.query_database(rel_query)
        return schema_text(node_props, rel_props, rels)

    def refresh_schema(self):
        self.schema = self.generate_schema()

    def get_system_message(self):
        return f"""
        Task: Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.
        Instructions:
        Use only the provided relationship types and properties.
        Do not use any other relationship types or properties that are not provided.
        If you cannot generate a Cypher statement based on the provided schema, explain the reason to the user.
        Schema:
        {self.schema}

        Note: Do not include any explanations or apologies in your responses.
        """

    def query_database(self, neo4j_query, params={}):
        with self.driver.session() as session:
            result = session.run(neo4j_query, params)
            output = [r.values() for r in result]
            output.insert(0, result.keys())
            return output

    def construct_cypher(self, question, history=None):
        messages = [
            {"role": "system", "content": self.get_system_message()},
            {"role": "user", "content": question},
        ]
        # Used for Cypher healing flows
        if history:
            messages.extend(history)

        
        completions = openai.ChatCompletion.create(
            model="gpt-4",
            temperature=0.0,
            max_tokens=1000,
            messages=messages
        )

        # cypher_generation_chain = LLMChain(llm=llm)
        # cypher_generation_chain.run({
        #     "question": "",
        #     "schema": "",
        # })
        
        return completions.choices[0].message.content

    def run(self, question, history=None, retry=True):
        # Construct Cypher statement
        cypher = self.construct_cypher(question, history)
        print(cypher)
        try:
            return self.query_database(cypher)
        # Self-healing flow
        except CypherSyntaxError as e:
            # If out of retries
            if not retry:
              return "Invalid Cypher syntax"
        # Self-healing Cypher flow by
        # providing specific error to GPT-4
            print("Retrying")
            return self.run(
                question,
                [
                    {"role": "assistant", "content": cypher},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)} 
                        Give me a improved query that works without any explanations or apologies""",
                    },
                ],
                retry=False
            )

In [6]:
username="neo4j"
password="AStglmnxZLS4bYASbsGkrV8ILu3XGJDdaeKiIxdTBvs"
url="neo4j+s://6b215408.databases.neo4j.io"
database="neo4j"

In [7]:
import os

openai_key = os.environ["OPENAI_API_KEY"]

In [8]:
gds_db = Neo4jGPTQuery(
    url=url,
    user=username,
    password=password,
    # openai_api_key=openai_key,
)

In [9]:
gds_db.run("가격이 3만원대의 메뉴를 파는 맛집 추천해줘")

MATCH (r:Restaurant)-[:SERVES]->(m:Menu)
WHERE m.price >= 30000 AND m.price < 40000
RETURN r.name, r.address, r.phone, r.operation


[['r.name', 'r.address', 'r.phone', 'r.operation'],
 ['닥터로빈 시그니처 판교점',
  '경기 성남시 분당구 동판교로177번길 25 아브뉴프랑 1층 108호 닥터로빈',
  '031-781-3105',
  '21:00에 라스트오더'],
 ['다이호시', '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', '0507-1425-8322', '12:00에 영업 시작'],
 ['다이호시', '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', '0507-1425-8322', '12:00에 영업 시작'],
 ['다이호시', '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', '0507-1425-8322', '12:00에 영업 시작'],
 ['다이호시', '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', '0507-1425-8322', '12:00에 영업 시작'],
 ['다이호시', '경기 성남시 분당구 판교역로10번길 6 1층 다이호시', '0507-1425-8322', '12:00에 영업 시작'],
 ['차알',
  '경기 성남시 분당구 동판교로 177 25번지 2층 214, 215호(아브뉴프랑)',
  '0507-1408-7679',
  '15:00에 브레이크타임'],
 ['차알',
  '경기 성남시 분당구 동판교로 177 25번지 2층 214, 215호(아브뉴프랑)',
  '0507-1408-7679',
  '15:00에 브레이크타임'],
 ['차알',
  '경기 성남시 분당구 동판교로 177 25번지 2층 214, 215호(아브뉴프랑)',
  '0507-1408-7679',
  '15:00에 브레이크타임'],
 ['제이스팟', '경기 성남시 분당구 판교역로 152 알파돔타워3동 3층', '0507-1344-7179', '15:00에 브레이크타임'],
 ['옥된장 서현점',
  '경기 성남시 분당구 황새울로335번길 5 1층 106호',
  '0507-1348-7522',
  '15:00에 브

In [12]:
gds_db.run("가격이 3만원대의 메뉴를 팔고 주차가 가능하며 반려동물을 데려갈수 있는 맛집 추천해줘")

I'm sorry, but the provided schema does not contain information about the price range of the menu, parking availability, or pet-friendly services. Therefore, it's not possible to generate a Cypher query based on your request.
Retrying
I'm sorry, but the provided schema does not contain information about the price range of the menu, parking availability, or pet-friendly services. Therefore, it's not possible to generate a Cypher query based on your request.


'Invalid Cypher syntax'